In [ ]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')
import createFolder as cF
### Import python libraries
from pyhdf.SD import SD, SDC
from pyhdf.HDF import *
from pyhdf.VS import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid

import pprint
import zipfile

import glob
import os
%matplotlib inline

In [ ]:
sourcefolder = '../../Data/CloudSat/zip.downloads/'

Product_name = ['GEOPROF']


In [ ]:
ym = [201611, 201612, 201701, 201702, 201703]


In [ ]:
savefig = 1

https://www.science-emergence.com/Articles/How-to-read-a-MODIS-HDF-file-using-python-/

https://www.science-emergence.com/Articles/How-to-read-CloudSat-2B-GEOPROF-GRANULE-HDF4-file-using-python-and-pyhdf-/

In [ ]:
stn_lat      = 59.81
stn_lon      = 7.21

lower_lat = stn_lat -1.
upper_lat = stn_lat +1.

left_lon = stn_lon -1.
right_lon = stn_lon +1.


http://www.trondkristiansen.com/?page_id=846

Since the ETOPO1 file is quite large (445MB) I added a function to the script that cuts out only the area of interest from the ETOPO1 file. This saves space and time (see function: findSubsetIndices).

In [ ]:
def findSubsetIndices(min_lat,max_lat,min_lon,max_lon,lats,lons):
    
    """Array to store the results returned from the function"""
    res=np.zeros((4),dtype=np.float64)
    minLon=min_lon; maxLon=max_lon
    
    distances1 = []; distances2 = []
    indices=[]; index=1
    
    for point in lats:
        s1 = max_lat-point # (vector subtract)
        s2 = min_lat-point # (vector subtract)
        distances1.append((np.dot(s1, s1), point, index))
        distances2.append((np.dot(s2, s2), point, index-1))
        index=index+1
        
    distances1.sort()
    distances2.sort()
    indices.append(distances1[0])
    indices.append(distances2[0])
    
    distances1 = []; distances2 = []; index=1
   
    for point in lons:
        s1 = maxLon-point # (vector subtract)
        s2 = minLon-point # (vector subtract)
        distances1.append((np.dot(s1, s1), point, index))
        distances2.append((np.dot(s2, s2), point, index-1))
        index=index+1
        
    distances1.sort()
    distances2.sort()
    indices.append(distances1[0])
    indices.append(distances2[0])
    
    """ Save final product: max_lat_indices,min_lat_indices,max_lon_indices,min_lon_indices"""
    minJ=indices[1][2] # min_lat_indices
    maxJ=indices[0][2]
    minI=indices[3][2]
    maxI=indices[2][2] # max_lon_indices
    
    res[0]=minI; res[1]=maxI; res[2]=minJ; res[3]=maxJ;
    return res

In [ ]:
for year_month in ym:
    targetfolder = '../../Data/CloudSat/%s/%s/' %(Product_name[0],year_month)
    os.chdir(targetfolder)
 

    if year_month == 201611:
        granule = np.arange(56028,56357)
    if year_month == 201612:
        granule = np.arange(56357,56806)
    if year_month == 201701:
        granule = np.arange(56816,57260)
    if year_month == 201702:
        granule = np.arange(57260,57668)
    if year_month == 201703:
        granule = np.arange(57668,58119)

    for i in granule:
        for file in glob.glob('*%s_CS_2B-GEOPROF_GRANULE_P_R04_E06.hdf' %(i)):


### Read latitudes and longitudes, profile time
            f = HDF(file, SDC.READ) 
            vs = f.vstart()

            data_info_list = vs.vdatainfo()


            vdata_lat = vs.attach('Latitude')
            vdata_long = vs.attach('Longitude')

            vdata_profile_time = vs.attach('Profile_time')



            lat = vdata_lat[:]
            long = vdata_long[:]

            profile_time = vdata_profile_time[:]


       
        
            vdata_lat.detach() # "close" the vdata
            vdata_long.detach() # "close" the vdata
            vdata_profile_time.detach()
            vs.end() # terminate the vdata interface
            f.close()




# Create a basemap to show the data
            fontsize = 16
            plt.figure(figsize=(15,5))
            m = Basemap(projection='cyl',llcrnrlat=lower_lat,urcrnrlat=upper_lat, 
                        llcrnrlon=left_lon,urcrnrlon=right_lon,resolution='c')
            m.drawcoastlines()
            m.fillcontinents(color='tan',lake_color='lightblue',zorder=1)
            m.drawmapboundary(fill_color='lightblue',zorder=0)
            m.drawmeridians(np.arange(np.round(left_lon),right_lon,1),labels=[0,0,0,1],fontsize=fontsize)
            m.drawparallels(np.arange(np.round(lower_lat),upper_lat,1),labels=[1,1,0,1],fontsize=fontsize)
            x,y = m(long,lat)


            m.scatter(x,y,fontsize,marker='o',color='b',zorder=2,label='Trajectory')
            m.scatter(stn_lon,stn_lat,10*fontsize,'r', marker="^",zorder=2)
            plt.text(stn_lon+0.1, stn_lat+0.1, 'Haukeliseter',fontsize=fontsize,
                     fontweight='bold', ha='left',va='top',color='k')
        
            plt.legend(loc='upper left',prop={'size': fontsize})
        
            split_file = file.split('_')
        

            plt.title("Cloudsat %s_%s" %(split_file[0],split_file[1]),fontsize=fontsize)

            if savefig == 1:
                if lower_lat == stn_lat -1.:
                    fig_dir = '../../../../Figures/cloudsat_trajectory/%s/2deg/' %(year_month)
                if lower_lat == stn_lat -2.:
                    fig_dir = '../../../../Figures/cloudsat_trajectory/%s/4deg/' %(year_month)
                if lower_lat == stn_lat -5.:
                    fig_dir = '../../../../Figures/cloudsat_trajectory/%s/10deg/' %(year_month)
                fig_name = '%s_%s.png' %(split_file[0],split_file[1])    
                cF.createFolder(fig_dir)
                plt.savefig(fig_dir+fig_name, bbox_inches='tight', dpi=200)
                print('figure saved:', fig_dir+fig_name)
            else:
                plt.show()
            plt.close()
    
    os.chdir('../../../../Python/CloudSat')
